In [1]:
from deepseek_vl2.models import DeepseekVLV2Processor, DeepseekVLV2ForCausalLM
from transformers import AutoModelForCausalLM
from pycocotools.coco import COCO
from pycocoevalcap.eval import COCOEvalCap
from huggingface_hub import login
import torch
import os
import json
from PIL import Image
from tqdm import tqdm
import numpy as np
from deepseek_vl2.utils.io import load_pil_images

Python version is above 3.10, patching the collections module.


In [2]:
# Verify numpy is available
print(f"Numpy version: {np.__version__}")
print(f"Torch version: {torch.__version__}")

Numpy version: 1.26.4
Torch version: 2.0.1+cu118


In [ ]:
your_hf_token = ""
login(token=your_hf_token)

In [ ]:
# Configuration
MODEL_PATH = "deepseek-ai/deepseek-vl2-tiny"
COCO_DIR = "Thesis/coco2014"
IMAGE_DIR = os.path.join(COCO_DIR, "val2014")
ANNOTATION_FILE = os.path.join(COCO_DIR, "annotations", "captions_val2014.json")
OUTPUT_FILE = "deepseek_vl2_tiny_coco_results.json"
BATCH_SIZE = 1
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Generation parameters
GENERATION_PARAMS = {
    "max_new_tokens": 50,
    "do_sample": False,  # Use greedy decoding for consistency
}

# Generation parameters
GENERATION_PARAMS = {
    "max_new_tokens": 50,
    "do_sample": False,
}

torch.cuda.empty_cache()
if torch.cuda.is_available():
    print(f"Free GPU memory: {torch.cuda.mem_get_info()[0] / 1024**3:.2f} GB")

Free GPU memory: 23.30 GB


In [5]:
def load_model_and_processor(model_path, device):
    """Load DeepSeek-VL2 model and processor."""
    print(f"Loading model: {model_path}")
    
    # Load processor
    vl_chat_processor = DeepseekVLV2Processor.from_pretrained(model_path)
    tokenizer = vl_chat_processor.tokenizer
    
    # Load model with bfloat16 for better performance
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        trust_remote_code=True,
        torch_dtype=torch.bfloat16,
        device_map="auto"
    ).eval()
    
    print(f"✓ Model loaded successfully")
    return model, vl_chat_processor, tokenizer

def generate_captions_batch(image_paths, model, processor, tokenizer, batch_size=2):
    """
    Generate captions for a batch of images using DeepSeek-VL2.
    
    Args:
        image_paths: List of image file paths
        model: DeepSeek-VL2 model
        processor: DeepSeek-VL2 processor
        tokenizer: Tokenizer from processor
        batch_size: Batch size for processing
        
    Returns:
        List of generated captions
    """
    captions = []
    
    for img_path in image_paths:
        try:
            # Create conversation format for single image with new role format
            conversation = [
                {
                    "role": "<|User|>",
                    "content": "<image>\nGenerate a detailed caption for this image in one sentence.",
                    "images": [img_path]
                },
                {
                    "role": "<|Assistant|>",
                    "content": ""
                }
            ]
            
            # Load image
            pil_images = load_pil_images(conversation)
            
            # Prepare inputs with system prompt (can be empty)
            prepare_inputs = processor(
                conversations=conversation,
                images=pil_images,
                force_batchify=True,
                system_prompt=""  # Empty system prompt for captioning task
            ).to(model.device)
            
            # Get image embeddings
            inputs_embeds = model.prepare_inputs_embeds(**prepare_inputs)
            
            # Generate response - CORRECTED: call generate() directly on model
            outputs = model.generate(
                inputs_embeds=inputs_embeds,
                attention_mask=prepare_inputs.attention_mask,
                pad_token_id=tokenizer.eos_token_id,
                bos_token_id=tokenizer.bos_token_id,
                eos_token_id=tokenizer.eos_token_id,
                max_new_tokens=GENERATION_PARAMS["max_new_tokens"],
                do_sample=GENERATION_PARAMS["do_sample"],
                use_cache=True
            )
            
            # Decode the response
            answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
            
            # Clean up the caption
            caption = answer.strip()
            # Remove the prompt part if it's included in the response
            if "Generate a detailed caption for this image in one sentence." in caption:
                caption = caption.split("Generate a detailed caption for this image in one sentence.")[-1].strip()
            
            captions.append(caption)
            
        except Exception as e:
            print(f"Error processing {img_path}: {e}")
            captions.append("Error generating caption")
    
    return captions

def evaluate_coco(annotation_file, results_file):
    """Evaluate results using pycocoevalcap."""
    print("\n" + "="*60)
    print("COCO Evaluation Results")
    print("="*60)
    
    # Load COCO API
    coco = COCO(annotation_file)
    coco_results = coco.loadRes(results_file)
    
    # Create evaluator
    coco_eval = COCOEvalCap(coco, coco_results)
    
    # Run evaluation
    coco_eval.evaluate()
    
    # Print scores
    print("\nMetrics:")
    for metric, score in coco_eval.eval.items():
        print(f"  {metric:10s}: {score:.4f}")
    
    return coco_eval.eval

def to_serializable(obj):
    """Convert numpy types to Python natives for JSON serialization."""
    if isinstance(obj, np.generic):
        return obj.item()
    raise TypeError(f"Type {type(obj)} not serializable")

In [6]:

# Load model and processor
model, processor, tokenizer = load_model_and_processor(MODEL_PATH, DEVICE)

Loading model: deepseek-ai/deepseek-vl2-tiny


/home/kezouke/Thesis/deepseek_vl2_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Add pad token = ['<｜▁pad▁｜>'] to the tokenizer
<｜▁pad▁｜>:2
Add image token = ['<image>'] to the tokenizer
<image>:128815
Add grounding-related tokens = ['<|ref|>', '<|/ref|>', '<|det|>', '<|/det|>', '<|grounding|>'] to the tokenizer with input_ids
<|ref|>:128816
<|/ref|>:128817
<|det|>:128818
<|/det|>:128819
<|grounding|>:128820
Add chat tokens = ['<|User|>', '<|Assistant|>'] to the tokenizer with input_ids
<|User|>:128821
<|Assistant|>:128822

✓ Model loaded successfully


In [7]:
# Load COCO annotations
print(f"\nLoading COCO annotations from {ANNOTATION_FILE}")
coco = COCO(ANNOTATION_FILE)
img_ids = coco.getImgIds()
print(f"Total images: {len(img_ids)}")
print(f"Generating captions for {len(img_ids)} images...")
print(f"Batch size: {BATCH_SIZE}\n")

results = []
image_data_list = []

# Prepare image data
for img_id in img_ids:
    img_info = coco.loadImgs(img_id)[0]
    image_path = os.path.join(IMAGE_DIR, img_info['file_name'])
    image_data_list.append((img_id, image_path))



Loading COCO annotations from /home/kezouke/Thesis/coco2014/annotations/captions_val2014.json
loading annotations into memory...
Done (t=0.57s)
creating index...
index created!
Total images: 40504
Generating captions for 40504 images...
Batch size: 1



In [8]:
print("Testing on first batch...")
test_batch_size = min(BATCH_SIZE, len(image_data_list))
test_paths = [image_data_list[i][1] for i in range(test_batch_size)]
test_captions = generate_captions_batch(test_paths, model, processor, tokenizer, BATCH_SIZE)
print(f"Sample captions from first batch:")
for i, caption in enumerate(test_captions[:3]):
    print(f"  {i+1}. {caption}")

Testing on first batch...


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/kezouke/Thesis/deepseek_vl2_env/lib/python3.10/site-packages/torch/nn/modules/conv.py:459: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


Sample captions from first batch:
  1. A person wearing a red shirt and helmet rides a motorcycle along a dirt path with mountains in the background.


In [9]:
# Process all images in batches
for batch_idx in tqdm(range(0, len(image_data_list), BATCH_SIZE), desc="Processing batches"):
    batch_data = image_data_list[batch_idx:batch_idx+BATCH_SIZE]
    img_ids_batch = [x[0] for x in batch_data]
    paths_batch = [x[1] for x in batch_data]
    
    # Generate captions for batch
    captions = generate_captions_batch(
        paths_batch, 
        model, 
        processor,
        tokenizer,
        batch_size=len(paths_batch)
    )
    
    # Add to results
    for img_id, caption in zip(img_ids_batch, captions):
        results.append({
            "image_id": img_id,
            "caption": caption
        })

Processing batches: 100%|██████████| 40504/40504 [3:48:15<00:00,  2.96it/s]  


In [10]:
# Save results
print(f"\nSaving results to {OUTPUT_FILE}")
with open(OUTPUT_FILE, 'w') as f:
    json.dump(results, f)
print("✓ Results saved")



Saving results to deepseek_vl2_tiny_coco_results.json
✓ Results saved


In [11]:

# Evaluate with COCO metrics
eval_scores = evaluate_coco(ANNOTATION_FILE, OUTPUT_FILE)



COCO Evaluation Results
loading annotations into memory...
Done (t=0.12s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.03s)
creating index...
index created!
tokenization...


PTBTokenizer tokenized 2492309 tokens at 3492052.51 tokens per second.
PTBTokenizer tokenized 748400 tokens at 3750857.54 tokens per second.


setting up scorers...
Progress: 384.5M / 384.5M (100.0%)
Extracting stanford-corenlp-3.6.0 ...
Done.
computing Bleu score...
{'testlen': 647476, 'reflen': 502458, 'guess': [647476, 606972, 566468, 525964], 'correct': [364049, 159351, 64706, 24608]}
ratio: 1.2886171580510186
Bleu_1: 0.562
Bleu_2: 0.384
Bleu_3: 0.256
Bleu_4: 0.168
computing METEOR score...
METEOR: 0.272
computing Rouge score...
ROUGE_L: 0.463
computing CIDEr score...
CIDEr: 0.549
computing SPICE score...


Parsing reference captions
Initiating Stanford parsing pipeline
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.TokenizerAnnotator - TokenizerAnnotator: No tokenizer type provided. Defaulting to PTBTokenizer.
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator parse
[main] INFO edu.stanford.nlp.parser.common.ParserGrammar - Loading parser from serialized file edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz ... 
done [0.2 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
Loading classifier from edu/stanford/nlp/models/ner/english.all.3class.distsim.crf.ser.gz ... done [0.5 sec].
Loading classifier from edu/stanford/nlp/models/ner/english.muc.7class.distsim.crf.ser.gz ... done [0.2 sec].
Loading classif

SPICE evaluation took: 12.84 min
SPICE: 0.222

Metrics:
  Bleu_1    : 0.5623
  Bleu_2    : 0.3842
  Bleu_3    : 0.2564
  Bleu_4    : 0.1676
  METEOR    : 0.2720
  ROUGE_L   : 0.4633
  CIDEr     : 0.5490
  SPICE     : 0.2219


In [12]:

# Save scores
scores_file = OUTPUT_FILE.replace('.json', '_scores.json')
with open(scores_file, 'w') as f:
    json.dump(eval_scores, f, indent=2, default=to_serializable)

print(f"✓ Scores saved to {scores_file}")

✓ Scores saved to deepseek_vl2_tiny_coco_results_scores.json
